<a href="https://colab.research.google.com/github/davis689/binder/blob/master/Maxwell_Boltzmann.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sympy as sp
from sympy import oo, exp, sin, pi
import matplotlib.pyplot as plt
import numpy as np
sp.init_printing()


# Maxwell-Boltzmann distribution

In [ ]:
eps,m,T,K=sp.symbols('varepsilon,m,T,K',positive=True,real=True)
vx,vy,vz,fvx,fex,fv=sp.symbols('v_x,v_y,v_z,f_{v_x},f_x,f_v',real=True)
kb=sp.symbols('k_B',constant=True,positive=True,real=True)

#One dimension# 
We begin with the Boltzmann distribution in one dimension. $$f(\varepsilon_i)=\left(\frac{1}{\sum_{j=0}^{\infty} e^{-ɛ_j/kT} }\right)e^{-\varepsilon_i/kT}$$ The term in parentheses is just a number and since we'll be normalizing our expression later, let's simplify this as just $K$.

In [ ]:
mbx=sp.Eq(fvx,K*sp.exp(-eps/kb/T))
mbx

The energy levels we're going to consider here are the energies of motion, $\frac{1}{2} m v_x^2$. Let's substitute this into our expression.

In [ ]:
mbx=mbx.subs(eps,m*vx**2/2)
mbx

Since we're deriving an expression for the *fraction* of molecules, we can normalize this expression to determine an expression for $K$. Since the sum of fractions over the whole range of velocities will be unity, our expression will be $$\int^\infty_{-\infty}K e^{-\frac{m v_x^2}{2k_B T}}=1$$ For the purposes *sympy*, to solve for K we need the expression to be solved to be equal to zero so we'll move the 1 over to the left hand side of the equation.

Here we'll solve for the normalization coefficient and call it *norm*. We'll then substitute it for K in the Maxwell-Boltzmann expression.

In [ ]:
norm=sp.solve(sp.integrate(mbx.rhs,(vx,-oo,oo))-1,K)[0] # solve returns a list. There is only one solution so [0] will get it.
norm

In [ ]:
mbx=mbx.subs(K,norm) # substitute the normalization coefficient into the distribution
mbx

To make sure this expression is indeed normalized, we can integrate it over the whole range and we should get 1. 

In [ ]:
sp.integrate(mbx.rhs,(vx,-oo,oo))

One disadvange of using the computer to do our math is that the final form might not match what we would do if we did the work on paper. In the case of the Maxwell-Boltzmann distribution, we can combine terms under one square root and obtain $$f_{v_x} = \sqrt{\dfrac{m}{2 \pi k_B T}} e^ {-\frac{m v_x^2}{2 k_B T}}$$

#Three dimensions#
To change to three dimensions, we realize that there is no real difference between molecules moving in any of the three dimensions so the $v_y$ function should look the same except with $v_y$ substituted for $v_x$. The overall distribution function is just the product of the three one-dimenional functions.

In [ ]:
mb3d=sp.Eq(fv,mbx.rhs*mbx.subs(vx,vy).rhs*mbx.subs(vx,vz).rhs)
mb3d

Of course, if we're looking at motion in three dimensions, we probably are much more interested in velocity in any direction rather than the three components of velocity along the axes. We can substitute $v^2=v_x^2+v_y^2+v_z^2$. 

In [ ]:
v=sp.symbols('v',real=True,positive=True)
mb3d.subs(vx**2+vy**2+vz**2,v**2)

In [ ]:
sp.powsimp(mb3d.subs(vx**2+vy**2+vz**2,v**2),force=True)

It's not clear that we can make the substitution that we want for $v$ in sympy. We'll have to do it another way. 

Since $v^2=v_x^2+v_y^2+v_z^2$ and since there's no difference between the values of $v_x$, $v_y$, and $v_z$, we can just write $v^2=3 v_x^2$ or $v_x=v/\sqrt{3}$.

In [ ]:
theta,phi=sp.symbols('theta, phi')
sp.integrate(sp.integrate(v**2*sp.sin(theta),(theta,0,sp.pi)),(phi,0,2*sp.pi))

In [ ]:
mb3d=sp.Eq(fv,mb3d.rhs.subs(vx,v/sp.sqrt(3)).subs(vy,v/sp.sqrt(3)).subs(vz,v/sp.sqrt(3))*4*sp.pi*v**2)
mb3d

In [ ]:
from sympy.plotting import plot
p=plot(mb3d.rhs.subs(kb,8.314).subs(T,298).subs(m,0.004).evalf(),(v,0,5000),show=False) # helium at 298 K
p.extend(plot(mb3d.rhs.subs(kb,8.314).subs(T,298).subs(m,0.040).evalf(),(v,0,5000),show=False,line_color='r')) #argon at 298 K
p.show()


In [ ]:
pp=plot(mb3d.rhs.subs(kb,8.314).subs(T,298).subs(m,0.004).evalf(),(v,0,5000),show=False) # helium at 298 K
pp.extend(plot(mb3d.rhs.subs(kb,8.314).subs(T,298).subs(m,0.040).evalf(),(v,0,5000),show=False,line_color='r')) #argon at 298 K
pp.show()

In [ ]:
He=sp.lambdify(v,mb3d.rhs.subs(kb,8.314).subs(T,298).subs(m,0.004).evalf())
Ar=sp.lambdify(v,mb3d.rhs.subs(kb,8.314).subs(T,298).subs(m,0.040).evalf())
speed=np.linspace(0,5000,200)

In [ ]:
plt.figure(figsize=(18,3))
plt.plot(speed,Ar(speed) )
plt.plot(speed,He(speed))
plt.xticks([])
plt.yticks([])
plt.savefig('mbbanner.jpg')
plt.show()

In [ ]:
fig,(ax1,ax2)=plt.subplots(1,2,sharey=True,figsize=(15,2))
ax1.plot(speed,He(speed))
ax2.plot(speed,Ar(speed))

In [ ]:
p1d=plot(mbx.rhs.subs(kb,8.314).subs(T,298).subs(m,0.004).evalf(),(vx,-5000,5000),show=False) # He at 298 K
p1d.extend(plot(mbx.rhs.subs(kb,8.314).subs(T,298).subs(m,0.040).evalf(),(vx,-5000,5000),show=False,line_color='r')) # Ar at 298 K
p1d.show()

Make a set of graphs for Ar at 298 K vs Ar at 500 K.

# Averages
Now that we a distribution function that gives the fraction of molecules going each speed, we can use it to get average values.

To obtain averages we need to weight each possible speed by the fraction of molecules going that speed and add up the results. In other words, $$\left<v\right> = \int^{oo}_0v f(v)dv$$

In [ ]:
v_avg=  sp.integrate(mb3d.rhs*v,(v,0,oo)) # average v
v_avg

A similar technique can get the average squared (or cubed) speed.

In [ ]:
sp.integrate(mb3d.rhs*v**2,(v,0,oo)) # average v**2

In [ ]:
sp.integrate(mb3d.rhs*v**3,(v,0,oo)) # average v**3

In [ ]:
sp.integrate(mb3d.rhs*m*v**2/2,(v,0,oo)) # average kinetic energy

Averge speed along the $x-$axis...

In [ ]:
vx_avg=sp.integrate(mbx.rhs*vx,(vx,-oo,oo)) # average v on x-axis
vx_avg

Average speed along the positive $x-$axis...

In [ ]:
vx_avg_plus=sp.integrate(mbx.rhs*vx,(vx,0,oo)) # average v on + x-axis
vx_avg_plus

In [ ]:
vx_sqrd=sp.integrate(mbx.rhs*vx**2,(vx,-oo,oo)) # average squared on x-axis
vx_sqrd

In [ ]:
Ek_avg=sp.integrate(mbx.rhs*vx**2*m/2,(vx,-oo,oo)) # average kinetic energy on x-axis
Ek_avg

# Collision Frequncy


In [ ]:
cs,V,N1,N2=sp.symbols('sigma, V, N_1,N_2',nonnegative=True,real=True)
from scipy.constants import Avogadro, Boltzmann
N_A=Avogadro
k_b=Boltzmann

In [ ]:
Z1=cs*v_avg*N1/V
Z1

In [ ]:
round(Z1.subs(N1,N_A).subs(V,.024).subs(T,298).subs(cs,pi*140e-12**2).subs(kb,k_b).subs(m,0.040/N_A).evalf(),2)

In [ ]:
Z2=cs*v_avg*N2/V
Z2

In [ ]:
round(Z2.subs(N2,N_A).subs(V,.024).subs(T,298).subs(cs,pi*140e-12**2).subs(kb,k_b).subs(m,0.040/N_A).evalf(),2)

In [ ]:
vr=v_avg*sp.integrate(sp.cos(theta),(theta,0,2*pi))
vr

It's actually not the average velocity that we want in this application. We need the average *relative* velocity. The average relative velocity is the difference between the velocity vectors of the two molecules. $\vec{v_{rel}}=\vec{v_1}-\vec{v_2}$ The magnitude of the relative velocity can be obtained from the dot product of the relative velocity with itself. $$v_{rel}=\sqrt{\vec{v_{rel}}\cdot\vec{v_{rel}}}=\sqrt{\left(\vec{v_1}-\vec{v_2}\right)\cdot\left(\vec{v_1}-\vec{v_2}\right)}=\sqrt{\vec{v_1}^2-2\vec{v_1}\cdot\vec{v_2}+\vec{v_2}^2}$$
For random, uncorrelated velocity vectors, the $\vec{v_1}\cdot\vec{v_2}=0$ so $$\vec{v_{rel}}=\sqrt{\vec{v_1}^2+\vec{v_2}^2}$$  Since the two velocity vectors are the same $$\vec{v_{rel}}=\sqrt{2\vec{v_1}^2}=\sqrt{2}\left<v\right>$$

In [ ]:
Z1=cs*v_avg*N1/V*sp.sqrt(2)
Z1

In [ ]:
Z2=cs*v_avg*N2/V*sp.sqrt(2)
Z2

In [ ]:
print(round(Z2.subs(N2,6e23).subs(V,.024).subs(T,298).subs(cs,pi*140e-12**2).subs(kb,1.38e-23).subs(m,0.040/6e23).evalf(),2),"1/s")

In [ ]:
Z11=N1/V*Z1/2
Z11

In [ ]:
print(round(Z11.subs(N2,6e23).subs(N1,6e23).subs(V,.024).subs(T,298).subs(cs,pi*140e-12**2).subs(kb,1.38e-23).subs(m,0.040/6e23).evalf(),2),"1/s")

# Mean Free Path

In [ ]:
mfp,p1=sp.symbols('lambda,p_1')
expr2=sp.Eq(mfp,v_avg/Z1)
expr2.subs(V/N1,kb*T/p1)

In [ ]:
sp.Eq(mfp,expr2.rhs.subs(V/N1,kb*T/p1).subs(T,298).subs(p1,100000).subs(kb,1.38e-23).subs(cs,pi*140e-12**2).evalf())

In [ ]:
expr2.subs(V/N1,kb*T/p1).subs(T,298).subs(p1,100000).subs(kb,1.38e-23).subs(cs,pi*140e-12**2).evalf()